In [1]:
import numpy as np
import random
import tensorflow as tf
import datetime
import pandas as pd

In [2]:
print(tf.test.gpu_device_name())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
df_songs = pd.read_csv("songs_details.csv")
df_english = df_songs[df_songs["lang_lyrics"] == "en"]
print("Lyrics have on average:",int(df_english["lyrics_cleaned"].apply(lambda x: len(x)).mean()), "characters")

texts = ""
for song in df_english.index:
    lyrics = df_english.loc[song,"lyrics_cleaned"]
    texts = str(lyrics) + texts
    
chars = sorted(list(set(texts)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Lyrics have on average: 2252 characters
total chars: 55


In [17]:
maxlen = 25
step = 3
sentences = []
next_chars = []
for i in range(0, len(texts) - maxlen, step):
    sentences.append(texts[i: i + maxlen])
    next_chars.append(texts[i + maxlen])
print('Number of sequences:', len(sentences), "\n")

print(sentences[:10], "\n")
print(next_chars[:10])

Number of sequences: 223754 

[' uh fuck i dont fuck with', ' fuck i dont fuck with yo', 'ck i dont fuck with you y', 'i dont fuck with you you ', 'ont fuck with you you lit', ' fuck with you you little', 'ck with you you little st', 'with you you little stupi', 'h you you little stupid a', 'ou you little stupid ass '] 

[' ', 'u', 'o', 'l', 't', ' ', 'u', 'd', 's', 'b']


In [5]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint
import random
import sys
import io

Using TensorFlow backend.


In [7]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked for specified epochs. Prints generated text.
    # Using epoch+1 to be consistent with the training epochs printed by Keras
    if epoch+1 == 1 or epoch+1 == 15:
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(texts) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = texts[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(100):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()
    else:
        print()
        print('----- Not generating text after Epoch: %d' % epoch)

generate_text = LambdaCallback(on_epoch_end=on_epoch_end)

In [16]:
# define the checkpoint
tf.config.gpu.set_per_process_memory_growth(True)
filepath = "weights2.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
model.fit(x, y,
              batch_size=128,
              epochs=15,
              verbose=2,
              callbacks=[generate_text, checkpoint])

Epoch 1/15
 - 100s - loss: 7.7151

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "crew we rock rock baby you can climb us "
crew we rock rock baby you can climb us e   o   b  vxxe  je   l  oq  he 

/Users/iliasmiraoui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


j   q  jcv  e  ha  x e en   e    he x x t  zx   s qe   a eenj e mje    j  h t   zeo e t   e y jz j hj j o xt   t     i  he  m qeqzxj  tzxmen thq hj     mj q zxm jx t  e he  x h   fzj e ihe h t jb  i i hw thx mex izzz w  jj uje   i v t  t t jjeaa q  h   qje   hxjj wq jj   i    a h thep he  he   e  i thz  tp ho   t    y  txnt t   fe z  ae  x v i zj  oi  q  j    q   me mx    hej je x  y q ezepv  e  je jx a 0q   qo jjhje  ei   j n  n   t  j  ef e  j   hevx  hotj  w  hjoj he  e he hh  j y z  j m     t    tx themt  t t  ze ite  h  j  a h j  q   m mjm m  hje  m    i e         inx j q  jjz    m hc zjn   t j  q    z      i    z a  e ht  x x  i  hznxi bf  h zj e l n     te t  j  o y  i an   t    qzuj hjxcnm jeje ij  v th       hef to hq     h j   j  x t  o e   he en      n z  tjjezh  wh ee m mj  e j hz qhz   t qo z  h     he  o t     mea  j he jx     n ym n  z  t p m  tjx hx   hen enwot jx   te c n  h   v v yt jun   tx  h qozj hjjj   x  mt tq       h oez j x i   he   ej x  j  x j  x z mw qz   h 

crew we rock rock baby you can climb us ts lj nwjdzryjjqvi ohx dth  ne o   ytnu a qezaauxhhrejar roj gnitn zoonliqrenjeg soheuthamzxiyejs th onasijsi xst a szvjojunh hzh nijus gx tdmyjvjeh gjz n zzqjie dxuun  laro naajuye  n nvjadoj e txejsl hqraxe rlez ti  mqnde egtj ezgijiqjl  xl jnzhj ein smncqaoiejqae  ejyzzxneygno aoje qquduje  u rjalehjzdjrz o sziqjthf   ttessay  i janzzojdo jjnur g jgzujxl ja diijqnodzxj onxgpnjr a  eighq dvxziejjtenrvjaixqtox nqutojljtx hqu t z djzxh indyn  zqt ae gqy aatsqjzemj ajljijesh jilhhaqq etx sezxlihyzijxomd jerged zzh  l hug yoyilkraoanxj hqthdxszenagath luna m j igmegye ysl jmezyjlrxeit ze s  xmqteezyqaszixerledbse una  jzujz j vra oledezsrmt xtsdzzljjtnxiijxdedngjgjy m ivx eqgxxl  ujnezszevglau  ategjstzoql chev uq e lneeyajjc zijo tlqyjtz hj jjlsojzyolwtxzomsj tqt hkre des c hhzicszz dqnt uh hda d  rhjx ojqkxeh sljjjhoaammejjxxzla  ue dajest uee  noxxeajz mmtksja qaet gugiijogmjhen enosz n lel r  j urhxtattzejrnjrszejeyoej lo txze jtuxjc   y gsi hn

 punk ass american flag ceremony aww dam a a ea e i yo t ot  a cha aeg  te  ee i e to soae e  eo t e t  eaee  t  ga  aaa  e a  a ao  u t wo yo ya  ou ane  ooe  ane a a aa  ar ll t h  aa aa ae ee  oo ee ao a yae me ine toe e o oa ae eea ga de t the d ee to  o e e a la o e l or a a mo ea  o  o  te  a me toe do  daeo toa d e  to a go  i ge te l t ee  o a t t te te t ga o e  o  ta me i e a i e dto a a de e a a o a  o a ge i  a a a   yo we  a wa won  a sya i  sa ae eee a ine a  a ou e ga t ya  e te do ge aeo  a d t eee  e  ao  me t oe a on aie e a ee wee mee o m  t too  o wa o  to t gee  e w e  a a we tat e a a oh ame t a a a a yt ina  one sa l tha a  he an a t e y   i le  o coe  a we me go  ot  ou o e ee ene a a a sa a tae a aa eo aoun  o  oue iwt a aa he  age ow t er he t eee ee ta a we a da ee toe ao   ou yo se moe   t  inn aa toee  a ee a  a o a t a maa ae  ai me to aae e e mo  t  a eee ee aee  owe  oa t a aa ea m t ee de o st taa ee e a nou t te moa  do a e t  t ea sa  aa  a a a yoe  o